# ADX and RSI Trendline Analysis

This notebook provides simplified functions to draw trendlines based on ADX and RSI indicators,
showing convergence and divergence between high-to-high and low-to-low points.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

In [ ]:
# Load dataset created by prepare_dataset.ipynb
df = pd.read_parquet('../data/dataset.parquet')
print(f"Loaded dataset with {len(df)} rows")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

In [ ]:
def find_local_extrema(series, window=5, mode='high'):
    """Find local maxima or minima in a time series.
    
    Args:
        series: Time series data
        window: Window size to look before and after a point
        mode: 'high' for maxima, 'low' for minima
        
    Returns:
        List of indices where extrema are found
    """
    indices = []
    values = []
    
    # Skip edge cases where we don't have enough points for the window
    for i in range(window, len(series)-window):
        window_slice = series[i-window:i+window+1]
        if mode == 'high' and series[i] == max(window_slice):
            indices.append(i)
            values.append(series[i])
        elif mode == 'low' and series[i] == min(window_slice):
            indices.append(i)
            values.append(series[i])
    
    return indices, values

In [ ]:
def pair_extrema_by_time(price_indices, indicator_indices, max_time_gap=10):
    """Pair price extrema with nearest indicator extrema within a time window.
    
    Args:
        price_indices: Indices of price extrema
        indicator_indices: Indices of indicator extrema
        max_time_gap: Maximum allowed index difference between pairs
        
    Returns:
        List of paired indices (price_idx, indicator_idx)
    """
    pairs = []
    
    for p_idx in price_indices:
        # Find the closest indicator extrema within the allowed time gap
        candidates = [i_idx for i_idx in indicator_indices 
                     if abs(i_idx - p_idx) <= max_time_gap]
        
        if candidates:
            # Select the closest indicator extrema
            closest_idx = min(candidates, key=lambda x: abs(x - p_idx))
            pairs.append((p_idx, closest_idx))
    
    return pairs

In [ ]:
def analyze_convergence_divergence(df, extrema_pairs, price_col, indicator_col, mode='high'):
    """Analyze if consecutive extrema pairs show convergence or divergence.
    
    Args:
        df: DataFrame with price and indicator data
        extrema_pairs: List of paired indices (price_idx, indicator_idx)
        price_col: Column name for price data
        indicator_col: Column name for indicator data
        mode: 'high' for peaks, 'low' for troughs
        
    Returns:
        List of dictionaries with trendline information and convergence/divergence type
    """
    results = []
    
    # Need at least two pairs to analyze trendlines
    if len(extrema_pairs) < 2:
        return results
    
    for i in range(len(extrema_pairs)-1):
        # Get current and next pair
        curr_price_idx, curr_ind_idx = extrema_pairs[i]
        next_price_idx, next_ind_idx = extrema_pairs[i+1]
        
        # Calculate slopes
        price_slope = (df[price_col].iloc[next_price_idx] - df[price_col].iloc[curr_price_idx]) / (next_price_idx - curr_price_idx)
        ind_slope = (df[indicator_col].iloc[next_ind_idx] - df[indicator_col].iloc[curr_ind_idx]) / (next_ind_idx - curr_ind_idx)
        
        # Determine pattern type
        pattern_type = 'neutral'
        if mode == 'high':
            # For highs: price up + indicator down = bearish divergence
            if price_slope > 0 and ind_slope < 0:
                pattern_type = 'bearish_divergence'
            # For highs: price down + indicator up = bullish divergence
            elif price_slope < 0 and ind_slope > 0:
                pattern_type = 'bullish_divergence'
            elif price_slope * ind_slope > 0:
                pattern_type = 'convergence'
        else:  # mode == 'low'
            # For lows: price down + indicator up = bullish divergence
            if price_slope < 0 and ind_slope > 0:
                pattern_type = 'bullish_divergence'
            # For lows: price up + indicator down = bearish divergence
            elif price_slope > 0 and ind_slope < 0:
                pattern_type = 'bearish_divergence'
            elif price_slope * ind_slope > 0:
                pattern_type = 'convergence'
        
        results.append({
            'price_start': (curr_price_idx, df[price_col].iloc[curr_price_idx]),
            'price_end': (next_price_idx, df[price_col].iloc[next_price_idx]),
            'indicator_start': (curr_ind_idx, df[indicator_col].iloc[curr_ind_idx]),
            'indicator_end': (next_ind_idx, df[indicator_col].iloc[next_ind_idx]),
            'price_slope': price_slope,
            'indicator_slope': ind_slope,
            'pattern_type': pattern_type,
            'start_date': df['date'].iloc[min(curr_price_idx, curr_ind_idx)],
            'end_date': df['date'].iloc[max(next_price_idx, next_ind_idx)],
        })
    
    return results

In [ ]:
def plot_trendlines(df, analysis_results, price_col, indicator_col, title=None):
    """Plot price and indicator with trendlines showing convergence/divergence.
    
    Args:
        df: DataFrame with price and indicator data
        analysis_results: Results from analyze_convergence_divergence
        price_col: Column name for price data
        indicator_col: Column name for indicator data
        title: Optional title for the plot
    """
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.03, 
                        subplot_titles=[f'Price ({price_col})', f'Indicator ({indicator_col})'])
    
    # Plot price data
    fig.add_trace(
        go.Scatter(x=df['date'], y=df[price_col], name=price_col, line=dict(color='blue')),
        row=1, col=1
    )
    
    # Plot indicator data
    fig.add_trace(
        go.Scatter(x=df['date'], y=df[indicator_col], name=indicator_col, line=dict(color='purple')),
        row=2, col=1
    )
    
    # Plot trendlines with different colors based on pattern type
    colors = {
        'bearish_divergence': 'red',
        'bullish_divergence': 'green',
        'convergence': 'gray',
        'neutral': 'lightgray'
    }
    
    for result in analysis_results:
        color = colors.get(result['pattern_type'], 'lightgray')
        
        # Price trendline
        fig.add_trace(
            go.Scatter(
                x=[df['date'].iloc[result['price_start'][0]], df['date'].iloc[result['price_end'][0]]],
                y=[result['price_start'][1], result['price_end'][1]],
                mode='lines',
                line=dict(color=color, width=2, dash='solid'),
                name=f'{result["pattern_type"]} (Price)',
                showlegend=False
            ),
            row=1, col=1
        )
        
        # Price points
        fig.add_trace(
            go.Scatter(
                x=[df['date'].iloc[result['price_start'][0]], df['date'].iloc[result['price_end'][0]]],
                y=[result['price_start'][1], result['price_end'][1]],
                mode='markers',
                marker=dict(color=color, size=8),
                name=f'{result["pattern_type"]} (Price Points)',
                showlegend=False
            ),
            row=1, col=1
        )
        
        # Indicator trendline
        fig.add_trace(
            go.Scatter(
                x=[df['date'].iloc[result['indicator_start'][0]], df['date'].iloc[result['indicator_end'][0]]],
                y=[result['indicator_start'][1], result['indicator_end'][1]],
                mode='lines',
                line=dict(color=color, width=2, dash='solid'),
                name=f'{result["pattern_type"]} ({indicator_col})',
                showlegend=(result == analysis_results[0])  # Show only one legend item per type
            ),
            row=2, col=1
        )
        
        # Indicator points
        fig.add_trace(
            go.Scatter(
                x=[df['date'].iloc[result['indicator_start'][0]], df['date'].iloc[result['indicator_end'][0]]],
                y=[result['indicator_start'][1], result['indicator_end'][1]],
                mode='markers',
                marker=dict(color=color, size=8),
                name=f'{result["pattern_type"]} ({indicator_col} Points)',
                showlegend=False
            ),
            row=2, col=1
        )
    
    # Update layout
    fig.update_layout(
        title=title or f'Trendline Analysis: {price_col} vs {indicator_col}',
        height=800,
        width=1200,
        legend=dict(orientation='h', y=1.02, xanchor='right', x=1),
        xaxis_rangeslider_visible=False
    )
    
    fig.show()

In [ ]:
def analyze_indicator_trendlines(df, price_col, indicator_col, extrema_window=10, 
                             time_gap=15, mode='high', min_patterns=3, plot=True):
    """Complete analysis pipeline for finding and plotting trendline convergence/divergence.
    
    Args:
        df: DataFrame with price and indicator data
        price_col: Column name for price data
        indicator_col: Column name for indicator data
        extrema_window: Window size for finding local extrema
        time_gap: Maximum time gap between price and indicator extrema to be paired
        mode: 'high' for peaks, 'low' for troughs
        min_patterns: Minimum number of patterns to find before returning results
        plot: Whether to plot the results
        
    Returns:
        Analysis results from analyze_convergence_divergence
    """
    # Find local extrema
    price_indices, _ = find_local_extrema(df[price_col].values, window=extrema_window, mode=mode)
    indicator_indices, _ = find_local_extrema(df[indicator_col].values, window=extrema_window, mode=mode)
    
    # Pair extrema by time
    extrema_pairs = pair_extrema_by_time(price_indices, indicator_indices, max_time_gap=time_gap)
    
    # Analyze convergence/divergence
    results = analyze_convergence_divergence(df, extrema_pairs, price_col, indicator_col, mode=mode)
    
    # If we don't have enough patterns, try with a smaller window
    if len(results) < min_patterns and extrema_window > 3:
        return analyze_indicator_trendlines(
            df, price_col, indicator_col, 
            extrema_window=extrema_window-2, 
            time_gap=time_gap,
            mode=mode,
            min_patterns=min_patterns,
            plot=plot
        )
    
    # Plot if requested
    if plot and results:
        mode_str = "Highs" if mode == "high" else "Lows"
        plot_trendlines(df, results, price_col, indicator_col, 
                      title=f'{mode_str} Trendline Analysis: {price_col} vs {indicator_col}')
    
    return results

## Run Trendline Analysis for ADX and RSI

Let's analyze both high-to-high and low-to-low trendlines for ADX and RSI.

In [ ]:
# Filter data to a reasonable time period for clearer visualization
recent_df = df.iloc[-365:].copy()  # Last year of data

# Analyze ADX trendlines
adx_high_results = analyze_indicator_trendlines(
    recent_df, 'high', 'adx', extrema_window=7, time_gap=10, mode='high'
)

adx_low_results = analyze_indicator_trendlines(
    recent_df, 'low', 'adx', extrema_window=7, time_gap=10, mode='low'
)

In [ ]:
# Analyze RSI trendlines
rsi_high_results = analyze_indicator_trendlines(
    recent_df, 'high', 'rsi_14', extrema_window=7, time_gap=10, mode='high'
)

rsi_low_results = analyze_indicator_trendlines(
    recent_df, 'low', 'rsi_14', extrema_window=7, time_gap=10, mode='low'
)

## Summary of Detected Patterns

Let's summarize the convergence/divergence patterns we found.

In [ ]:
def summarize_patterns(results, pattern_name):
    """Create a summary of detected patterns."""
    if not results:
        print(f"No {pattern_name} patterns detected.")
        return
    
    pattern_counts = {}
    for result in results:
        pattern_type = result['pattern_type']
        pattern_counts[pattern_type] = pattern_counts.get(pattern_type, 0) + 1
    
    print(f"\n{pattern_name} - {len(results)} total patterns:")
    for pattern_type, count in pattern_counts.items():
        print(f"  {pattern_type}: {count} patterns")
        
    # Show the most recent patterns of each type
    print("\nMost recent patterns:")
    for pattern_type in pattern_counts.keys():
        type_results = [r for r in results if r['pattern_type'] == pattern_type]
        if type_results:
            latest = max(type_results, key=lambda x: x['end_date'])
            print(f"  {pattern_type}: {latest['start_date'].date()} to {latest['end_date'].date()}")

# Print summaries
summarize_patterns(adx_high_results, "ADX High-to-High")
summarize_patterns(adx_low_results, "ADX Low-to-Low")
summarize_patterns(rsi_high_results, "RSI High-to-High")
summarize_patterns(rsi_low_results, "RSI Low-to-Low")

In [ ]:
def find_strongest_signals(results, top_n=3):
    """Find the strongest divergence signals based on slope difference."""
    if not results:
        return []
    
    # Filter for divergence patterns only
    divergences = [r for r in results if 'divergence' in r['pattern_type']]
    
    # Calculate strength based on slope difference
    for div in divergences:
        div['strength'] = abs(div['price_slope'] - div['indicator_slope'])
    
    # Sort by strength
    strongest = sorted(divergences, key=lambda x: x['strength'], reverse=True)[:top_n]
    return strongest

# Find strongest divergence signals
strongest_adx = find_strongest_signals(adx_high_results + adx_low_results)
strongest_rsi = find_strongest_signals(rsi_high_results + rsi_low_results)

print("Strongest ADX divergence signals:")
for i, signal in enumerate(strongest_adx, 1):
    print(f"{i}. {signal['pattern_type']} from {signal['start_date'].date()} to {signal['end_date'].date()}")

print("\nStrongest RSI divergence signals:")
for i, signal in enumerate(strongest_rsi, 1):
    print(f"{i}. {signal['pattern_type']} from {signal['start_date'].date()} to {signal['end_date'].date()}")